# Neutron density and exposure

In [ ]:
# ppn run results are supposed to be in directory ppn_dir+'ppn_run_test_'+case,
# where case is 'Sakurai', '2ndpeak', or 'weak'

# select a case from the following ones:
case = '2ndpeak'
#case = 'Sakurai'
#case = 'weak'

### Initializing the notebook

In [ ]:
%pylab ipympl
from nugridpy import ppn
from nugridpy import utils
from nugridpy import astronomy as ast
from scipy import integrate
from astropy import constants as c

In [ ]:
# constants in SI units
NAv = c.N_A.value
kB  = c.k_B.value # Boltzmann_constant             
mn =  c.m_n.value # neutron mass
oneyear=utils.constants.one_year # seconds in one year
print ("Avogadro number:",NAv,"\nBoltzmann constant:",kB,"\nneutron mass:",mn,"\nseconds per year:",oneyear)

In [ ]:
case = '2ndpeak'
rundir = 'master-result'
#rundir = 'I135ng_up3'

In [ ]:
f0=ppn.xtime('../'+rundir)   # an x-time instance

In [ ]:
# prepare quantities to plot

cycles=f0.get("cycle")
first_cycle = int(min(cycles))
last_cycle = int(max(cycles))

xn  = f0.get("NEUT")
age = f0.get("time")
rho = f0.get("rho")
t9  = f0.get("t9")

age_s=age*oneyear
T = t9*1e9
vnth = sqrt(2.*kB*T/mn)            # neutron thermal velocity
den = 1e6*rho                      # density in kg/ccm; rho is in g/ccm
Nn = xn*NAv*den *1e-6              # neutron density in particles per ccm
logNn = log10(Nn)
fint = (Nn*1e6)*vnth
tau=integrate.cumtrapz(fint,age_s) # integrate neutron exposure
tau=tau*1e-31                      # neutron exposure in millibarn^-1; millibarn=1e-31 m^2

## Neutron density

In [ ]:
ifig=1;close(ifig);figure(ifig)
plot(age_s,log10(Nn))
ylim(7,17)
xlabel("time (s)")
ylabel("$N_\mathrm{n}$")
title(case+' '+rundir)

## Neutron exposure

In [ ]:
ifig=2;close(ifig);figure(ifig)
plot(age_s[1:],tau)
xlabel("time (s)")
ylabel("$ \\tau [\mathrm{mbarn}^{-1}]$")
title(case+' '+rundir)

## Neutron exposure and neutron density

In [ ]:
fig, ax1 = plt.subplots()
name1 = '$N_\mathrm{n}$'
lns1 = ax1.semilogy(age_s/60., Nn, color=utils.linestylecb(0)[2], linestyle=utils.linestylecb(0)[0],\
            label=name1)
ax1.set_xlabel('time (min)')
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel(name1+'$\ [\mathrm{cm}^{-3}]$')
ax1.tick_params('y')
ax1.set_ylim(1e11,10.**ceil(log10(max(Nn))))

ax2 = ax1.twinx()
name2 = '$\\tau$'
lns2 = ax2.semilogy(age_s[1:]/60., tau, color=utils.linestylecb(1)[2], linestyle=utils.linestylecb(1)[0],\
            label=name2)
ax2.set_ylabel(name2+'$\ [\mathrm{mbarn}^{-1}]$')
ax2.tick_params('y')
ax2.set_ylim(1e-1,None)

# added these three lines
lns = lns1+lns2
labs = [l.get_label() for l in lns]
ax2.legend(lns, labs, loc=5, frameon=False)

title(case)

fig.tight_layout()
show()